In [16]:
# import libs 
import torch 
import torch.nn as nn 
import torch.optim as optim 
from torch.optim import lr_scheduler
import numpy as np 
import torchvision 
from torchvision import datasets,transforms,models 
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt 
import time 
import os 
import copy 
import glob 

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [9]:
# Transforms
transformer = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5 , 0.5 , 0.5 ],[0.5 , 0.5 , 0.5 ]),
])

In [10]:
# set directory paths 
base_dir = "basedata"
training = "training"
validationString = "validation"

# training ,validation locations 
training_path = os.path.join(base_dir,training)
test_path = os.path.join(base_dir,"testing")

In [11]:
# DataLoader 
train_loader = DataLoader(
    torchvision.datasets.ImageFolder(training_path , transform = transformer),
    batch_size = 12 , shuffle= True
)
test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path , transform = transformer),
    batch_size = 12 , shuffle= True
)

In [12]:
model = models.resnet18(pretrained = True)

/home/anirudh/miniconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/anirudh/miniconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/anirudh/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [07:36<00:00, 103kB/s]   


In [14]:
# get class
classes = os.listdir(training_path)

In [15]:

# Set HyperParameters 
num_epochs = 3 
batch_size = 4 
learning_rate = 0.001
# Tune this later based on performance 

In [17]:
# calculate testing_training , size 
train_count = len(glob.glob(training_path+'/**/*.JPG'))
test_count = len(glob.glob(test_path+'/**/*.JPG'))

In [18]:
num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs,len(classes))
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.001)

In [20]:
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i , (images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)


        # Forward pass 
        outputs = model(images)
        loss = criterion(outputs,labels)

        # Backward pass 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    # if (i+1) % 2 == 0:
    print('Epoch {} , Loss : {}'.format(epoch,loss.item())) 

print('Finished Training')

Epoch 0 , Loss : 3.6536941528320312
Epoch 1 , Loss : 3.658282518386841
Epoch 2 , Loss : 3.684581756591797
Finished Training


In [22]:
# For testing 
with torch.no_grad():
    n_correct = 0 
    n_samples = 0 
    n_class_correct = [0 for i in range(len(classes))]
    n_class_samples = [0 for i in range(len(classes))]

    for images,labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        _ , predicted = torch.max(outputs , 1)
        n_samples +=labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]

            if (label == pred):
                n_class_correct[label] +=1
            n_class_samples[label]+=1
    
    accuracy = 100.0* n_correct/n_samples 
    print('Accuracy  : {} % '.format(accuracy))

    for i in range(len(classes)):
        if n_class_samples[i] == 0:
            print('Didnt get the sample in test ')
        else:
            acc = 100*n_class_correct[i]/n_class_samples[i]
            print('Accuracy of {}  : {} %'.format(classes[i],acc))
            

Accuracy  : 76.3975155279503 % 
Accuracy of 023  : 100.0 %
Accuracy of 004  : 100.0 %
Accuracy of 033  : 100.0 %
Accuracy of 037  : 100.0 %
Accuracy of 010  : 33.333333333333336 %
Accuracy of 013  : 100.0 %
Accuracy of 035  : 100.0 %
Accuracy of 039  : 66.66666666666667 %
Accuracy of 001  : 50.0 %
Accuracy of 025  : 100.0 %
Accuracy of 041  : 100.0 %
Accuracy of 029  : 33.333333333333336 %
Accuracy of 016  : 100.0 %
Accuracy of 009  : 75.0 %
Accuracy of 026  : 33.333333333333336 %
Accuracy of 028  : 100.0 %
Accuracy of 022  : 100.0 %
Didnt get the sample in test 
Accuracy of 027  : 100.0 %
Accuracy of 030  : 33.333333333333336 %
Didnt get the sample in test 
Accuracy of 034  : 100.0 %
Accuracy of 021  : 100.0 %
Accuracy of 024  : 50.0 %
Accuracy of 005  : 100.0 %
Accuracy of 031  : 100.0 %
Accuracy of 007  : 100.0 %
Didnt get the sample in test 
Accuracy of 017  : 50.0 %
Accuracy of 014  : 75.0 %
Accuracy of 020  : 80.0 %
Accuracy of 036  : 100.0 %
Accuracy of 038  : 100.0 %
Accuracy o